In [1]:
import glob
import pandas as pd

files = glob.glob('../evaluate_results/*.csv', recursive=True)
files[0:3]

["../evaluate_results/SVC-split_x_y_split_with_bag_of_words-{'C': [1.0], 'kernel': ['linear'], 'probability': [True]}.csv",
 '../evaluate_results/LogisticRegression-split_x_y_word2vec-{}.csv',
 "../evaluate_results/SVC-split_x_y_with_random_matrix-{'C': [10000.0], 'gamma': [1e-05], 'kernel': ['rbf'], 'probability': [True]}.csv"]

In [2]:
frames = [pd.read_csv(file, index_col=[0]) for file in files]
data = pd.concat(frames)

data.head(3)

/home/paulo/git/RBM/venv/lib/python3.6/site-packages/ipykernel-5.1.0-py3.6.egg/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,column,evaluation_method,mean_fit_time,mean_score_time,mean_test_accuracy,mean_test_f1_weighted,mean_test_mrr,mean_test_precision_weighted,mean_test_recall_weighted,mean_train_accuracy,...,std_test_accuracy,std_test_f1_weighted,std_test_mrr,std_test_precision_weighted,std_test_recall_weighted,std_train_accuracy,std_train_f1_weighted,std_train_mrr,std_train_precision_weighted,std_train_recall_weighted
0,0,SVC-split_x_y_split_with_bag_of_words-{'C': [1...,64.911475,7.881548,0.462875,0.406968,0.557450,0.381154,0.462875,0.902133,...,0.039303,0.044690,0.041692,0.051037,0.039303,0.005510,0.005874,0.011590,0.004499,0.005510
0,1,SVC-split_x_y_split_with_bag_of_words-{'C': [1...,38.250833,8.366286,0.258031,0.218628,0.340697,0.211047,0.258031,0.878278,...,0.013711,0.016269,0.028479,0.015840,0.013711,0.004514,0.004334,0.015360,0.003028,0.004514
0,2,SVC-split_x_y_split_with_bag_of_words-{'C': [1...,34.270873,8.885417,0.246972,0.213528,0.329407,0.209244,0.246972,0.884802,...,0.031531,0.031829,0.033653,0.037627,0.031531,0.002000,0.002278,0.014315,0.002540,0.002000


In [3]:
'''
# Old
data_train = data[(data['evaluate_method'] == 'Accuracy') & (data['evaluation'] == 'train')]
data_test = data[(data['evaluate_method'] == 'Accuracy') & (data['evaluation'] == 'test')]

values_columns = [f"{i}" for i in range(0, 6)]
group_columns = ['model', 'evaluate_method']

data_grouped = data_test.groupby(group_columns)[values_columns].mean()
data_grouped

data_grouped.T.mean().sort_values(ascending=False)

pd.read_csv('../other_models/data_search_parameter/SVC-bag_of_words_gaussian_random_projection 2.csv', index_col=[0]).mean()
'''

'\n# Old\ndata_train = data[(data[\'evaluate_method\'] == \'Accuracy\') & (data[\'evaluation\'] == \'train\')]\ndata_test = data[(data[\'evaluate_method\'] == \'Accuracy\') & (data[\'evaluation\'] == \'test\')]\n\nvalues_columns = [f"{i}" for i in range(0, 6)]\ngroup_columns = [\'model\', \'evaluate_method\']\n\ndata_grouped = data_test.groupby(group_columns)[values_columns].mean()\ndata_grouped\n\ndata_grouped.T.mean().sort_values(ascending=False)\n\npd.read_csv(\'../other_models/data_search_parameter/SVC-bag_of_words_gaussian_random_projection 2.csv\', index_col=[0]).mean()\n'

In [4]:
def compute_mean(data, column):
    mean_by_column = data.pivot_table(index='evaluation_method', columns='column', values=column)
    mean_general = mean_by_column.T.mean().sort_values(ascending=False).to_frame()
    
    index_columns = ['classifier', 'x_transform', 'params']
    
    mean_general = mean_general.reset_index()
    data = mean_general['evaluation_method'].str.split('-', n=2, expand=True)
    data.columns = index_columns
    data[column] = mean_general[0]
    data = data.set_index(index_columns)
    
    return mean_by_column, data

In [5]:
mean_by_column, mean_general = compute_mean(data, 'mean_test_accuracy')
mean_by_column

column,0,1,2,3,4,5
evaluation_method,,,,,,
"KNeighborsClassifier-split_x_y-{'n_neighbors': [1], 'algorithm': ['brute'], 'metric': ['hamming']}",0.391785,0.223275,0.182201,0.253818,0.347025,0.507109
"KNeighborsClassifier-split_x_y-{'n_neighbors': [20], 'algorithm': ['brute'], 'metric': ['hamming']}",0.425487,0.196419,0.182201,0.288573,0.367562,0.542917
"KNeighborsClassifier-split_x_y_word2vec-{'n_neighbors': [20], 'algorithm': ['brute'], 'metric': ['hamming']}",0.425487,0.196419,0.182201,0.288573,0.367562,0.542917
LogisticRegression-split_x_y-{},0.348078,0.146919,0.149026,0.259610,0.303318,0.461295
LogisticRegression-split_x_y_split_with_bag_of_words-{},0.460769,0.253291,0.242233,0.315429,0.403370,0.551869
LogisticRegression-split_x_y_split_with_bag_of_words_and_projection-{},0.399684,0.180095,0.173249,0.267509,0.363349,0.509215
LogisticRegression-split_x_y_word2vec-{},0.350184,0.169036,0.134281,0.224855,0.304371,0.467088
"MLPClassifier-split_x_y-{'hidden_layer_sizes': [5], 'max_iter': [500]}",0.339652,0.133228,0.139547,0.230648,0.307004,0.464455
"MLPClassifier-split_x_y_split_with_bag_of_words-{'hidden_layer_sizes': [5], 'max_iter': [500]}",0.398104,0.177462,0.174829,0.250132,0.324381,0.494471


In [6]:
mean_general

mean_test_accuracy
classifier             x_transform                                      params                                                                
LogisticRegression     split_x_y_split_with_bag_of_words                {}                                                            0.371160
SVC                    split_x_y_split_with_bag_of_words                {'C': [1.0], 'kernel': ['linear'], 'probability...            0.365368
                                                                        {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...            0.364753
KNeighborsClassifier   split_x_y_word2vec                               {'n_neighbors': [20], 'algorithm': ['brute'], '...            0.333860
                       split_x_y                                        {'n_neighbors': [20], 'algorithm': ['brute'], '...            0.333860
SVC                    split_x_y                                        {'C': [1.0], 'gamma': [0.01], 'kernel': ['rbf']...            0.325434
                       split_x_y_split_with_bag_of_words_and_projection {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...            0.322187
KNeighborsClassifier   split_x_y                                        {'n_neighbors': [1], 'algorithm': ['brute'], 'm...            0.317536
LogisticRegression     split_x_y_split_with_bag_of_words_and_projection {}                                                            0.315517
SVC                    split_x_y_split_with_projection                  {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...            0.312445
                       split_x_y_with_random_matrix                     {'C': [1.0], 'gamma': [0.01], 'kernel': ['rbf']...            0.311831
MLPClassifier          split_x_y_split_with_bag_of_words                {'hidden_layer_sizes': [5], 'max_iter': [500]}                0.303230
SVC                    split_x_y_with_random_matrix                     {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...            0.302966
                       split_x_y                                        {'C': [100.0], 'gamma': [1e-05], 'kernel': ['rb...            0.300070
MLPClassifier          split_x_y_split_with_bag_of_words_and_projection {'hidden_layer_sizes': [5], 'max_iter': [500]}                0.298841
SVC                    split_x_y_word2vec                               {'C': [1.0], 'kernel': ['linear'], 'probability...            0.281639
LogisticRegression     split_x_y                                        {}                                                            0.278041
                       split_x_y_word2vec                               {}                                                            0.274969
MLPClassifier          split_x_y_word2vec                               {'hidden_layer_sizes': [5], 'max_iter': [500]}                0.270142
                       split_x_y                                        {'hidden_layer_sizes': [5], 'max_iter': [500]}                0.269089
SVC                    split_x_y_word2vec                               {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...            0.260664
RBMAlreadyTrainedModel split_x_y                                        {}                                                            0.059242

In [7]:
mean_data = data.groupby(['evaluation_method', 'column']).mean()

mean_by_column, mean_general = compute_mean(data, 'mean_test_mrr')
mean_by_column

column,0,1,2,3,4,5
evaluation_method,,,,,,
"KNeighborsClassifier-split_x_y-{'n_neighbors': [1], 'algorithm': ['brute'], 'metric': ['hamming']}",0.396984,0.229914,0.189191,0.260195,0.352606,0.511322
"KNeighborsClassifier-split_x_y-{'n_neighbors': [20], 'algorithm': ['brute'], 'metric': ['hamming']}",0.496285,0.243572,0.236975,0.347549,0.428584,0.604682
"KNeighborsClassifier-split_x_y_word2vec-{'n_neighbors': [20], 'algorithm': ['brute'], 'metric': ['hamming']}",0.496285,0.243572,0.236975,0.347549,0.428584,0.604682
LogisticRegression-split_x_y-{},0.477765,0.249350,0.247648,0.347397,0.414744,0.573580
LogisticRegression-split_x_y_split_with_bag_of_words-{},0.568664,0.362998,0.353963,0.424270,0.512085,0.648352
LogisticRegression-split_x_y_split_with_bag_of_words_and_projection-{},0.510525,0.284190,0.277956,0.366688,0.462252,0.609224
LogisticRegression-split_x_y_word2vec-{},0.479859,0.265495,0.243506,0.321042,0.418778,0.583957
"MLPClassifier-split_x_y-{'hidden_layer_sizes': [5], 'max_iter': [500]}",0.471263,0.241708,0.233331,0.324013,0.411163,0.575771
"MLPClassifier-split_x_y_split_with_bag_of_words-{'hidden_layer_sizes': [5], 'max_iter': [500]}",0.494391,0.280110,0.273404,0.348842,0.427394,0.587177


In [8]:
mean_general

mean_test_mrr
classifier             x_transform                                      params                                                           
LogisticRegression     split_x_y_split_with_bag_of_words                {}                                                       0.478389
SVC                    split_x_y_split_with_bag_of_words                {'C': [1.0], 'kernel': ['linear'], 'probability...       0.463495
                                                                        {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...       0.461188
                       split_x_y_split_with_bag_of_words_and_projection {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...       0.422786
LogisticRegression     split_x_y_split_with_bag_of_words_and_projection {}                                                       0.418473
SVC                    split_x_y                                        {'C': [1.0], 'gamma': [0.01], 'kernel': ['rbf']...       0.410224
                       split_x_y_word2vec                               {'C': [1.0], 'kernel': ['linear'], 'probability...       0.408199
                       split_x_y_with_random_matrix                     {'C': [1.0], 'gamma': [0.01], 'kernel': ['rbf']...       0.403622
                       split_x_y                                        {'C': [100.0], 'gamma': [1e-05], 'kernel': ['rb...       0.403553
                       split_x_y_word2vec                               {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...       0.402696
MLPClassifier          split_x_y_split_with_bag_of_words                {'hidden_layer_sizes': [5], 'max_iter': [500]}           0.401886
                       split_x_y_split_with_bag_of_words_and_projection {'hidden_layer_sizes': [5], 'max_iter': [500]}           0.399692
SVC                    split_x_y_split_with_projection                  {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...       0.398436
                       split_x_y_with_random_matrix                     {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...       0.393391
KNeighborsClassifier   split_x_y                                        {'n_neighbors': [20], 'algorithm': ['brute'], '...       0.392941
                       split_x_y_word2vec                               {'n_neighbors': [20], 'algorithm': ['brute'], '...       0.392941
LogisticRegression     split_x_y_word2vec                               {}                                                       0.385440
                       split_x_y                                        {}                                                       0.385081
MLPClassifier          split_x_y_word2vec                               {'hidden_layer_sizes': [5], 'max_iter': [500]}           0.380103
                       split_x_y                                        {'hidden_layer_sizes': [5], 'max_iter': [500]}           0.376208
KNeighborsClassifier   split_x_y                                        {'n_neighbors': [1], 'algorithm': ['brute'], 'm...       0.323369
RBMAlreadyTrainedModel split_x_y                                        {}                                                       0.105710

In [9]:
prefix = 'mean_test'
columns = list(filter(lambda x: x.startswith(prefix), data.columns))

values = []

for column in columns:
    _, value = compute_mean(data, column)
    values.append(value)

fill_result = pd.concat(values, axis=1)
fill_result.sort_values('mean_test_accuracy', ascending=False)

mean_test_accuracy  \
classifier             x_transform                                      params                                                                   
LogisticRegression     split_x_y_split_with_bag_of_words                {}                                                            0.371160   
SVC                    split_x_y_split_with_bag_of_words                {'C': [1.0], 'kernel': ['linear'], 'probability...            0.365368   
                                                                        {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...            0.364753   
KNeighborsClassifier   split_x_y_word2vec                               {'n_neighbors': [20], 'algorithm': ['brute'], '...            0.333860   
                       split_x_y                                        {'n_neighbors': [20], 'algorithm': ['brute'], '...            0.333860   
SVC                    split_x_y                                        {'C': [1.0], 'gamma': [0.01], 'kernel': ['rbf']...            0.325434   
                       split_x_y_split_with_bag_of_words_and_projection {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...            0.322187   
KNeighborsClassifier   split_x_y                                        {'n_neighbors': [1], 'algorithm': ['brute'], 'm...            0.317536   
LogisticRegression     split_x_y_split_with_bag_of_words_and_projection {}                                                            0.315517   
SVC                    split_x_y_split_with_projection                  {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...            0.312445   
                       split_x_y_with_random_matrix                     {'C': [1.0], 'gamma': [0.01], 'kernel': ['rbf']...            0.311831   
MLPClassifier          split_x_y_split_with_bag_of_words                {'hidden_layer_sizes': [5], 'max_iter': [500]}                0.303230   
SVC                    split_x_y_with_random_matrix                     {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...            0.302966   
                       split_x_y                                        {'C': [100.0], 'gamma': [1e-05], 'kernel': ['rb...            0.300070   
MLPClassifier          split_x_y_split_with_bag_of_words_and_projection {'hidden_layer_sizes': [5], 'max_iter': [500]}                0.298841   
SVC                    split_x_y_word2vec                               {'C': [1.0], 'kernel': ['linear'], 'probability...            0.281639   
LogisticRegression     split_x_y                                        {}                                                            0.278041   
                       split_x_y_word2vec                               {}                                                            0.274969   
MLPClassifier          split_x_y_word2vec                               {'hidden_layer_sizes': [5], 'max_iter': [500]}                0.270142   
                       split_x_y                                        {'hidden_layer_sizes': [5], 'max_iter': [500]}                0.269089   
SVC                    split_x_y_word2vec                               {'C': [10000.0], 'gamma': [1e-05], 'kernel': ['...            0.260664   
RBMAlreadyTrainedModel split_x_y                                        {}                                                            0.059242   

                                                                                                                            mean_test_f1_weighted  \
classifier             x_transform                                      params                                                                      
LogisticRegression     split_x_y_split_with_bag_of_words                {}                                                               0.296240   
SVC                    split_x_y_split_with_bag_of_words                {'C': [1.0], 'kernel': ['linear'], 'probability...               0.326526   
                       